In [ ]:
"""
pymatviz requires Python >= 3.10
"""


In [1]:
"""
This code is adapted from https://github.com/janosh/pymatviz.
I've introduced a new parameter, scaling_factor, to enable dynamic sizing. 
I've submitted a pull request, but it may take some time to get approved.
"""


"""Periodic table plots powered by plotly."""

from __future__ import annotations

from collections.abc import Sequence
from typing import TYPE_CHECKING, Literal

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go

from pymatviz.enums import ElemCountMode
from pymatviz.process_data import count_elements
from pymatviz.utils import ElemValues, df_ptable


if TYPE_CHECKING:
    from collections.abc import Callable
    from typing import Any


def ptable_heatmap_plotly(
    values: ElemValues,
    *,
    count_mode: ElemCountMode = ElemCountMode.composition,
    colorscale: str | Sequence[str] | Sequence[tuple[float, str]] = "viridis",
    show_scale: bool = True,
    show_values: bool = True,
    heat_mode: Literal["value", "fraction", "percent"] = "value",
    fmt: str | None = None,
    hover_props: Sequence[str] | dict[str, str] | None = None,
    hover_data: dict[str, str | int | float] | pd.Series | None = None,
    font_colors: Sequence[str] = (),
    gap: float = 5,
    font_size: int | None = None,
    bg_color: str | None = None,
    nan_color: str = "#eff",
    color_bar: dict[str, Any] | None = None,
    cscale_range: tuple[float | None, float | None] = (None, None),
    exclude_elements: Sequence[str] = (),
    log: bool = False,
    fill_value: float | None = None,
    label_map: dict[str, str] | Callable[[str], str] | Literal[False] | None = None,
    border: dict[str, Any] | None | Literal[False] = None,
    scaling_factor: float = 1.0,
    **kwargs: Any,
) -> go.Figure:
    """Create a Plotly figure with an interactive heatmap of the periodic table.
    Supports hover tooltips with custom data or atomic reference data like
    electronegativity, atomic_radius, etc. See kwargs hover_data and hover_props, resp.

    Args:
        values (dict[str, int | float] | pd.Series | list[str]): Map from element
            symbols to heatmap values e.g. dict(Fe=2, O=3) or iterable of composition
            strings or Pymatgen composition objects.
        count_mode ("composition" | "fractional_composition" | "reduced_composition"):
            Reduce or normalize compositions before counting. See count_elements() for
            details. Only used when values is list of composition strings/objects.
        colorscale (str | list[str] | list[tuple[float, str]]): Color scale for heatmap.
            Defaults to "viridis". See plotly.com/python/builtin-colorscales for names
            of other builtin color scales. Note "YlGn" and px.colors.sequential.YlGn are
            equivalent. Custom scales are specified as ["blue", "red"] or
            [[0, "rgb(0,0,255)"], [0.5, "rgb(0,255,0)"], [1, "rgb(255,0,0)"]].
        show_scale (bool): Whether to show a bar for the color scale. Defaults to True.
        show_values (bool): Whether to show numbers on heatmap tiles. Defaults to True.
        heat_mode ("value" | "fraction" | "percent" | None): Whether to display heat
            values as is (value), normalized as a fraction of the total, as percentages
            or not at all (None). Defaults to "value".
            "fraction" and "percent" can be used to make the colors in different
            periodic table heatmap plots comparable.
        fmt (str): f-string format option for heat labels. Defaults to ".1%"
            (1 decimal place) if heat_mode="percent" else ".3g".
        hover_props (list[str] | dict[str, str]): Elemental properties to display in the
            hover tooltip. Can be a list of property names to display only the values
            themselves or a dict mapping names to what they should display as. E.g.
            dict(atomic_mass="atomic weight") will display as `"atomic weight = {x}"`.
            Defaults to None.
            Available properties are: symbol, row, column, name,
            atomic_number, atomic_mass, n_neutrons, n_protons, n_electrons, period,
            group, phase, radioactive, natural, metal, nonmetal, metalloid, type,
            atomic_radius, electronegativity, first_ionization, density, melting_point,
            boiling_point, number_of_isotopes, discoverer, year, specific_heat,
            n_shells, n_valence.
        hover_data (dict[str, str | int | float] | pd.Series): Map from element symbols
            to additional data to display in the hover tooltip. dict(Fe="this appears in
            the hover tooltip on a new line below the element name"). Defaults to None.
        font_colors (list[str]): One color name or two for [min_color, max_color].
            min_color is applied to annotations with heatmap values less than
            (max_val - min_val) / 2. Defaults to None, meaning auto-set to maximize
            contrast with color scale: white text for dark background and vice versa.
            swapped depending on the colorscale.
        gap (float): Gap in pixels between tiles of the periodic table. Defaults to 5.
        font_size (int): Element symbol and heat label text size. Any valid CSS size
            allowed. Defaults to automatic font size based on plot size. Element symbols
            will be bold and 1.5x this size.
        bg_color (str): Plot background color. Defaults to "rgba(0, 0, 0, 0)".
        color_bar (dict[str, Any]): Plotly colorbar properties documented at
            https://plotly.com/python/reference#heatmap-colorbar. Defaults to
            dict(orientation="h"). Commonly used keys are:
            - title: colorbar title
            - titleside: "top" | "bottom" | "right" | "left"
            - tickmode: "array" | "auto" | "linear" | "log" | "date" | "category"
            - tickvals: list of tick values
            - ticktext: list of tick labels
            - tickformat: f-string format option for tick labels
            - len: fraction of plot height or width depending on orientation
            - thickness: fraction of plot height or width depending on orientation
        nan_color (str): Fill color for element tiles with NaN values. Defaults to
            "#eff".
        cscale_range (tuple[float | None, float | None]): Colorbar range. Defaults to
            (None, None) meaning the range is automatically determined from the data.
        exclude_elements (list[str]): Elements to exclude from the heatmap. E.g. if
            oxygen overpowers everything, you can do exclude_elements=["O"].
            Defaults to ().
        log (bool): Whether to use a logarithmic color scale. Defaults to False.
            Piece of advice: colorscale="viridis" and log=True go well together.
        fill_value (float | None): Value to fill in for missing elements. Defaults to 0.
        label_map (dict[str, str] | Callable[[str], str] | None): Map heat values (after
            string formatting) to target strings. Set to False to disable. Defaults to
            dict.fromkeys((np.nan, None, "nan", "nan%"), "-") so as not to display "nan"
            for missing values.
        border (dict[str, Any]): Border properties for element tiles. Defaults to
            dict(width=1, color="gray"). Other allowed keys are arguments of go.Heatmap
            which is (mis-)used to draw the borders as a 2nd heatmap below the main one.
            Pass False to disable borders.
        scaling_factor (float): scaling factor for all figure layout
        **kwargs: Additional keyword arguments passed to
            plotly.figure_factory.create_annotated_heatmap().

    Returns:
        Figure: Plotly Figure object.
    """
    if log and heat_mode in ("fraction", "percent"):
        raise ValueError(
            "Combining log color scale and heat_mode='fraction'/'percent' unsupported"
        )
    if len(cscale_range) != 2:
        raise ValueError(f"{cscale_range=} should have length 2")

    if isinstance(colorscale, str | type(None)):
        colorscale = px.colors.get_colorscale(colorscale or "viridis")
    elif not isinstance(colorscale, Sequence) or not isinstance(
        colorscale[0], str | list | tuple
    ):
        raise TypeError(
            f"{colorscale=} should be string, list of strings or list of "
            "tuples(float, str)"
        )

    color_bar = color_bar or {}
    color_bar.setdefault("orientation", "h")
    # if values is a series with a name, use it as the colorbar title
    if isinstance(values, pd.Series) and values.name:
        color_bar.setdefault("title", values.name)

    values = count_elements(values, count_mode, exclude_elements, fill_value)

    if heat_mode in ("fraction", "percent"):
        # normalize heat values
        clean_vals = values.replace([np.inf, -np.inf], np.nan).dropna()
        # ignore inf values in sum() else all would be set to 0 by normalizing
        heat_value_element_map = values / clean_vals.sum()
    else:
        heat_value_element_map = values

    n_rows, n_columns = 10, 18
    # initialize tile text and hover tooltips to empty strings
    tile_texts, hover_texts = np.full([2, n_rows, n_columns], "", dtype=object)
    heatmap_values = np.full([n_rows, n_columns], np.nan)

    if label_map is None:
        # default to space string for None, np.nan and "nan". space is needed
        # for <br> in tile_text to work so all element symbols are vertically aligned
        label_map = dict.fromkeys([np.nan, None, "nan", "nan%"], "-")  # type: ignore[list-item]

    all_ints = all(isinstance(val, int) for val in values)
    counts_total = values.sum()
    for symbol, period, group, name, *_ in df_ptable.itertuples():
        # build table from bottom up so that period 1 becomes top row
        row = n_rows - period
        col = group - 1

        label = ""  # label (if not None) is placed below the element symbol
        if show_values:
            if symbol in exclude_elements:
                label = "excl."
            elif heat_val := heat_value_element_map.get(symbol):
                if heat_mode == "percent":
                    label = f"{heat_val:{fmt or '.1%'}}"
                else:
                    default_prec = ".1f" if heat_val < 100 else ",.0f"
                    if heat_val > 1e5:
                        default_prec = ".2g"
                    label = f"{heat_val:{fmt or default_prec}}".replace("e+0", "e")

            if callable(label_map):
                label = label_map(label)
            elif isinstance(label_map, dict):
                label = label_map.get(label, label)
        style = (
            f"font-weight: bold; font-size: {1.5 * (font_size or 12*scaling_factor)};"
        )
        tile_text = f"<span {style=}>{symbol}</span>"
        if show_values and label:
            tile_text += f"<br>{label}"

        tile_texts[row][col] = tile_text

        hover_text = name

        if heat_val := heat_value_element_map.get(symbol):
            if all_ints:
                # if all values are integers, values are likely element
                # counts, so makes sense to show count and percentage
                percentage = heat_val / counts_total
                hover_text += f"<br>Value: {heat_val} ({percentage:.2%})"
            elif heat_mode == "value":
                hover_text += f"<br>Value: {heat_val:.3g}"
            elif heat_mode in ("fraction", "percent") and (orig_val := values[symbol]):
                hover_text += f"<br>Percentage: {heat_val:.2%} ({orig_val:.3g})"

        if hover_data is not None and symbol in hover_data:
            hover_text += f"<br>{hover_data[symbol]}"

        if hover_props is not None:
            if unsupported_keys := set(hover_props) - set(df_ptable):
                raise ValueError(
                    f"Unsupported hover_props: {', '.join(unsupported_keys)}. Available"
                    f" keys are: {', '.join(df_ptable)}.\nNote that some keys have "
                    "missing values."
                )
            df_row = df_ptable.loc[symbol]
            if isinstance(hover_props, dict):
                for col_name, col_label in hover_props.items():
                    hover_text += f"<br>{col_label} = {df_row[col_name]}"
            elif isinstance(hover_props, list | tuple):
                hover_text += "<br>" + "<br>".join(
                    f"{col_name} = {df_row[col_name]}" for col_name in hover_props
                )
            else:
                raise ValueError(
                    f"hover_props must be dict or sequence of str, got {hover_props}"
                )

        hover_texts[row][col] = hover_text

        if symbol in exclude_elements:
            continue

        color_val = heat_value_element_map[symbol]
        if log and color_val > 0:
            color_val = np.log10(color_val)
        # until https://github.com/plotly/plotly.js/issues/975 is resolved, we need to
        # insert transparency (rgba0) at low end of colorscale (+1e-6) to not show any
        # colors on empty tiles of the periodic table
        heatmap_values[row][col] = color_val

    if isinstance(font_colors, str):
        font_colors = [font_colors]

    non_nan_values = [val for val in heatmap_values.flat if not np.isnan(val)]

    zmin = min(non_nan_values) if cscale_range[0] is None else cscale_range[0]
    zmax = max(non_nan_values) if cscale_range[1] is None else cscale_range[1]
    car_multiplier = 100 if heat_mode == "percent" else 1

    fig = ff.create_annotated_heatmap(
        car_multiplier * heatmap_values,
        annotation_text=tile_texts,
        text=hover_texts,
        showscale=show_scale,
        colorscale=colorscale,
        font_colors=font_colors or None,
        hoverinfo="text",
        xgap=gap,
        ygap=gap,
        zauto=False,  # Disable auto-scaling
        zmin=zmin * car_multiplier,
        zmax=zmax * car_multiplier,
        **kwargs,
    )

    # Add border heatmap
    if border is not False:
        border = border or {}
        border_color = border.pop("color", "darkgray")
        border_width = border.pop("width", 2)

        common_kwargs = dict(
            z=np.where(tile_texts, 1, np.nan), showscale=False, hoverinfo="none"
        )
        # misuse heatmap to add borders around all element tiles
        # 1st one adds the fill color for NaN element tiles, 2nd one adds the border
        fig.add_heatmap(
            **common_kwargs,
            colorscale=[nan_color, nan_color],
            xgap=gap,
            ygap=gap,
        )
        fig.add_heatmap(
            **common_kwargs,
            colorscale=[border_color, border_color],
            xgap=gap - border_width,
            ygap=gap - border_width,
            **border,
        )

        # reverse fig.data to place the border heatmap below the main heatmap
        fig.data = fig.data[::-1]

    fig.update_layout(
        margin=dict(l=10, r=10, t=10, b=10, pad=10),
        paper_bgcolor=bg_color,
        plot_bgcolor="rgba(0, 0, 0, 0)",
        xaxis=dict(zeroline=False, showgrid=False),
        yaxis=dict(zeroline=False, showgrid=False, scaleanchor="x"),
        font_size=font_size*scaling_factor if font_size else 12*scaling_factor,
        width=1000*scaling_factor,
        height=500*scaling_factor,
        title=dict(x=0.4, y=0.95),
    )

    if color_bar.get("orientation") == "h":
        dct = dict(x=0.4, y=0.75, titleside="top", len=0.4)
        color_bar = {**dct, **color_bar}
    else:  # make title vertical
        dct = dict(titleside="right", len=0.87)
        color_bar = {**dct, **color_bar}
        if title := color_bar.get("title"):
            # <br><br> to increase title offset
            color_bar["title"] = f"<br><br>{title}"

    if log:
        orig_min = np.floor(min(non_nan_values))
        orig_max = np.ceil(max(non_nan_values))
        tick_values = np.logspace(orig_min, orig_max, num=10, endpoint=True)

        tick_values = [round(val, -int(np.floor(np.log10(val)))) for val in tick_values]

        color_bar = dict(
            tickvals=np.log10(tick_values),
            ticktext=[f"{v * car_multiplier:.2g}" for v in tick_values],
            **color_bar,
        )

    # suffix % to colorbar title if heat_mode is "percent"
    if heat_mode == "percent" and (cbar_title := color_bar.get("title")):
        color_bar["title"] = f"{cbar_title} (%)"

    fig.update_traces(colorbar=dict(lenmode="fraction", thickness=15, **color_bar))

    return fig


In [60]:
elements_dict = {
    "Hydrogen": "H",
    "Helium": "He",
    "Lithium": "Li",
    "Beryllium": "Be",
    "Boron": "B",
    "Carbon": "C",
    "Nitrogen": "N",
    "Oxygen": "O",
    "Fluorine": "F",
    "Neon": "Ne",
    "Sodium": "Na",
    "Magnesium": "Mg",
    "Aluminum": "Al",
    "Silicon": "Si",
    "Phosphorus": "P",
    "Sulfur": "S",
    "Chlorine": "Cl",
    "Argon": "Ar",
    "Potassium": "K",
    "Calcium": "Ca",
    "Scandium": "Sc",
    "Titanium": "Ti",
    "Vanadium": "V",
    "Chromium": "Cr",
    "Manganese": "Mn",
    "Iron": "Fe",
    "Cobalt": "Co",
    "Nickel": "Ni",
    "Copper": "Cu",
    "Zinc": "Zn",
    "Gallium": "Ga",
    "Germanium": "Ge",
    "Arsenic": "As",
    "Selenium": "Se",
    "Bromine": "Br",
    "Krypton": "Kr",
    "Rubidium": "Rb",
    "Strontium": "Sr",
    "Yttrium": "Y",
    "Zirconium": "Zr",
    "Niobium": "Nb",
    "Molybdenum": "Mo",
    "Technetium": "Tc",
    "Ruthenium": "Ru",
    "Rhodium": "Rh",
    "Palladium": "Pd",
    "Silver": "Ag",
    "Cadmium": "Cd",
    "Indium": "In",
    "Tin": "Sn",
    "Antimony": "Sb",
    "Tellurium": "Te",
    "Iodine": "I",
    "Xenon": "Xe",
    "Cesium": "Cs",
    "Barium": "Ba",
    "Lanthanum": "La",
    "Cerium": "Ce",
    "Praseodymium": "Pr",
    "Neodymium": "Nd",
    "Promethium": "Pm",
    "Samarium": "Sm",
    "Europium": "Eu",
    "Gadolinium": "Gd",
    "Terbium": "Tb",
    "Dysprosium": "Dy",
    "Holmium": "Ho",
    "Erbium": "Er",
    "Thulium": "Tm",
    "Ytterbium": "Yb",
    "Lutetium": "Lu",
    "Hafnium": "Hf",
    "Tantalum": "Ta",
    "Tungsten": "W",
    "Rhenium": "Re",
    "Osmium": "Os",
    "Iridium": "Ir",
    "Platinum": "Pt",
    "Gold": "Au",
    "Mercury": "Hg",
    "Thallium": "Tl",
    "Lead": "Pb",
    "Bismuth": "Bi",
    "Polonium": "Po",
    "Astatine": "At",
    "Radon": "Rn",
    "Francium": "Fr",
    "Radium": "Ra",
    "Actinium": "Ac",
    "Thorium": "Th",
    "Protactinium": "Pa",
    "Uranium": "U",
    "Neptunium": "Np",
    "Plutonium": "Pu",
    "Americium": "Am",
    "Curium": "Cm",
    "Berkelium": "Bk",
    "Californium": "Cf",
    "Einsteinium": "Es",
    "Fermium": "Fm",
    "Mendelevium": "Md",
    "Nobelium": "No",
    "Lawrencium": "Lr",
    "Rutherfordium": "Rf",
    "Dubnium": "Db",
    "Seaborgium": "Sg",
    "Bohrium": "Bh",
    "Hassium": "Hs",
    "Meitnerium": "Mt",
    "Darmstadtium": "Ds",
    "Roentgenium": "Rg",
    "Copernicium": "Cn",
    "Nihonium": "Nh",
    "Flerovium": "Fl",
    "Moscovium": "Mc",
    "Livermorium": "Lv",
    "Tennessine": "Ts",
    "Oganesson": "Og"
}
empty_element_count = {ele:0 for ele in elements_dict.values()}

In [62]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd
import json

# from pymatviz import ptable_heatmap_plotly

# MP API endpoints that contain chemical element information
endpoint_list = [
    'absorption',
    'bonds',
    'dielectric',
    'elasticity',
    'electronic_structure_bandstructure', 
    'electronic_structure_dos',
    'electronic_structure',
    'insertion_electrodes',
    'magnetism',
    'oxidation_states',
    'piezoelectric',
    'summary', 
    'thermo',
]

app = Dash()
"""
app.layout = [
    html.H1(children='Materials Project Data Statistics', style={'textAlign':'center'}),
    dcc.Dropdown(
        options=endpoint_list, 
        value='absorption', 
        id='dropdown-selection'
    ),
    html.Div(
        children=[
            dcc.Graph(id='histogram', style={'width': '40%', 'display': 'inline-block', 'vertical-align': 'top'}),
            dcc.Graph(id='ptable_endpoint', style={'width': '55%', 'display': 'inline-block', 'vertical-align': 'top'}),
            dcc.Graph(id='ptable_endpoint_element', style={'width': '55%', 'display': 'inline-block', 'vertical-align': 'top'}),
        ])
    ]
"""
app.layout = [
    # title and dropdown
    html.Div([
        html.H1(
            children='Materials Project Data Statistics', style={'textAlign':'center'}
        ),
        dcc.Dropdown(
            options=endpoint_list, 
            value='absorption', 
            id='dropdown-selection'
        )
        ]
    ),  
    # figure
    html.Div([
        # Left figure: histogram
        html.Div([
            dcc.Graph(id='histogram')
            ], 
            style={'flex': '1', 'margin-right': '10px'}
        ),  # Left column, larger

        # Right column
        html.Div([
            # Top right figure
            html.Div([
                dcc.Graph(id='ptable_endpoint')
            ], 
            style={'flex': '1.5'}
            ),  # Top right

            # Bottom right figure
            html.Div([
                dcc.Graph(id='ptable_endpoint_element')
            ])  # Bottom right
        ], style={'flex': '1.5'})  # Right column, taller
    ], style={'display': 'flex'})] # Using flexbox for layout


@callback(
    Output('histogram', 'figure'),
    Output('ptable_endpoint', 'figure'),
    Input('dropdown-selection', 'value')
)
def update_fig_enpoint(end_point):
    
    global api_end_point
    api_end_point = end_point
    
    # histogram
    with open(f"./data/count.json", 'r') as json_file:
        endpoint_material_count = json.load(json_file)
    endpoint_count_df = pd.DataFrame(endpoint_material_count, index=[0]).T.reset_index()
    endpoint_count_df.columns = ['endpoint', 'count']
    endpoint_count_df.sort_values('count', inplace=True)
    colors = ['blue' if endpoint != end_point else 'red' for endpoint in endpoint_count_df['endpoint']]
    fig_hist = px.bar(
        endpoint_count_df, 
        x='endpoint', 
        y='count', 
        color=colors
    )
    fig_hist.update_layout(
        showlegend=False,
        margin=dict(l=0, r=0, t=20, b=20),
        yaxis=dict(type='log')
    )

    # chemical elements distribution
    with open(f"./data/count_ele_{end_point}.json", 'r') as json_file:
        element_count = json.load(json_file)
    fig_enpoint = ptable_heatmap_plotly(
            element_count,
            # hover_props=["atomic_number", "type"],
            hover_props=["name"],
            scaling_factor=0.5
        )
    
    return(fig_hist, fig_enpoint)


@callback(
    Output('ptable_endpoint_element', 'figure'),
    Input('ptable_endpoint', 'clickData'),
)
def update_fig_endpoint_element(clickData):
    if clickData:
        # Access the hover text from clickData
        print(clickData.keys())
        try:
            element_name = clickData['points'][0]['text'].split('<br>')[0]
        except:
            element_name = None
        print(element_name)
        if element_name:
            symbol = elements_dict[element_name]
            
            # get 
            with open(f"./data/elemental/count_ele_ele_{api_end_point}.json", 'r') as json_file:
                element_element_count = json.load(json_file)
            
            # chemical elements distribution on specific element-based
            fig_endpoint_element = ptable_heatmap_plotly(
                    element_element_count[symbol],
                    hover_props=["name"],
                    scaling_factor=0.5
                )
            return(fig_endpoint_element)

    fig_endpoint_element = ptable_heatmap_plotly(
        empty_element_count, 
        scaling_factor=0.5
    )
    return(fig_endpoint_element)
        

if __name__ == '__main__':
    app.run(debug=True)


dict_keys(['points'])
Calcium
